In [1]:
import pandas as pd
import numpy as np
import scipy as sp
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from jeffsearchcv import JeffRandomSearchCV
from frankenscorer import FrankenScorer

In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

In [4]:
X, y = load_breast_cancer(return_X_y=True)
clf = RandomForestClassifier()
params = {'n_estimators': sp.stats.randint(low=10, high=500),
          'max_depth':[None, 1, 2, 3, 4, 5, 10, 20]}
search = JeffRandomSearchCV(clf, params, scoring=FrankenScorer(), cv=5, n_iter=6)
search.fit(X, y)

JeffRandomSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=6, n_jobs=1,
          param_distributions={'max_depth': [None, 1, 2, 3, 4, 5, 10, 20], 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2e8f277f98>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True,
          scoring=<frankenscorer.FrankenScorer object at 0x7f2e8f277b00>,
          verbose=0)

In [8]:
from frankenscorer import extract_score_grid

In [12]:
test_score_grid = extract_score_grid(search)

In [42]:
test_score_grid.mean_assumed_f1_test

0    0.969681
1    0.957437
2    0.964098
3    0.964010
4    0.970969
5    0.964049
Name: mean_assumed_f1_test, dtype: float64

In [17]:
import copy

In [18]:
results = pd.DataFrame(copy.deepcopy(search.cv_results_))

In [19]:
splits = search.cv if search.cv is not None else 3
splits

5

In [21]:
score_data_test = results['split1_test_score_data'].iloc[0]
score_data_test['confusion_matrix_lab'].shape
score_data_test['labeled_f1'].shape

()

In [22]:
from collections import defaultdict

In [25]:
#create master_dict of scores
master_dict = {}
for row in range(len(results)):
    row_dict = defaultdict(dict)
    for split in range(splits):
        for tpe in ['test','train']:
            split_results = copy.deepcopy(results['split{}_{}_score_data'.format(str(split), tpe)].iloc[row])
            d = {}
            for k, v in split_results.items():
                new_key = "{}_{}{}".format(k,tpe,split)
                if hasattr(v, 'shape') and v.shape == (2, 2):
                    tn, fp, fn, tp = v.ravel()
                    d["tn_%s" % new_key] = tn
                    d["fp_%s" % new_key] = fp
                    d["fn_%s" % new_key] = fn
                    d["tp_%s" % new_key] = tp
                if FrankenScorer.score_index != k:
                    #don't include the "SCORE" score in the grid
                    d[new_key] = v
            row_dict[row].update(d)
    master_dict.update(row_dict)

In [24]:
#delete SCORE columns from
for row in range(len(master_dict)):
    for split in range(splits):
        for tpe in ['test','train']:
            try:
                del master_dict[row]["{}_{}{}".format(FrankenScorer.score_index,tpe,split)]
            except KeyError:
                pass

In [26]:
score_grid = pd.DataFrame.from_dict(master_dict, orient="index")

In [27]:
score_grid

,labeled_brier_train4,pr_one_unlabeled_test3,labeled_brier_test4,pu_score_test2,assumed_brier_test4,labeled_roc_auc_test4,assumed_f1beta10_test1,fn_confusion_matrix_un_test0,labeled_acc_test3,confusion_matrix_lab_train3,confusion_matrix_lab_test3,labeled_roc_auc_test0,fp_confusion_matrix_un_test1,labeled_avg_prec_train3,labeled_avg_prec_test1,confusion_matrix_un_test2,tp_confusion_matrix_un_test0,labeled_brier_test2,fn_confusion_matrix_un_train2,pu_score_test0,fn_confusion_matrix_lab_train0,assumed_f1_test3,tn_confusion_matrix_lab_test2,tp_confusion_matrix_un_test2,labeled_prec_test1,tn_confusion_matrix_un_test1,tn_confusion_matrix_lab_train0,assumed_f1_train2,fp_confusion_matrix_un_train3,fn_confusion_matrix_un_train1,tn_confusion_matrix_un_test4,tn_confusion_matrix_un_train3,assumed_f1beta10_test3,labeled_brier_train3,labeled_avg_prec_train2,labeled_roc_auc_train0,tp_confusion_matrix_un_test3,labeled_prec_test4,pr_one_unlabeled_train3,fn_confusion_matrix_un_test1,labeled_roc_auc_train1,fn_confusion_matrix_lab_test4,tp_confusion_matrix_un_train0,assumed_brier_train3,pr_one_unlabeled_test0,labeled_recall_test2,fp_confusion_matrix_un_test3,labeled_avg_prec_test0,assumed_brier_test2,tn_confusion_matrix_lab_test0,pu_score_train1,assumed_f1beta10_train4,labeled_avg_prec_test3,assumed_f1_train3,fn_confusion_matrix_lab_test3,tp_confusion_matrix_un_test1,tn_confusion_matrix_lab_train2,confusion_matrix_un_test4,pr_one_unlabeled_test1,assumed_brier_train4,labeled_prec_test0,assumed_brier_train2,labeled_acc_test0,tn_confusion_matrix_lab_train1,confusion_matrix_un_train2,labeled_brier_train1,fp_confusion_matrix_un_test2,assumed_brier_test0,confusion_matrix_lab_train0,assumed_f1_train1,fp_confusion_matrix_lab_test4,tp_confusion_matrix_lab_test1,fn_confusion_matrix_lab_test1,confusion_matrix_un_test0,fp_confusion_matrix_lab_test3,fn_confusion_matrix_un_train0,confusion_matrix_lab_train1,tp_confusion_matrix_un_train1,assumed_brier_train1,assumed_f1_train0,tn_confusion_matrix_un_test2,fn_confusion_matrix_un_train4,confusion_matrix_un_test3,labeled_f1_test4,labeled_roc_auc_test3,pu_score_train3,labeled_prec_test3,pu_score_test1,labeled_brier_test0,pr_one_unlabeled_test4,labeled_acc_test1,fn_confusion_matrix_lab_train4,fn_confusion_matrix_un_test2,fp_confusion_matrix_un_train2,assumed_f1beta10_train1,labeled_avg_prec_test2,labeled_brier_test3,labeled_prec_train3,assumed_f1beta10_test4,assumed_brier_test3,confusion_matrix_lab_test0,labeled_roc_auc_train4,labeled_recall_test4,labeled_roc_auc_train3,pr_one_unlabeled_train0,pr_one_unlabeled_train4,fp_confusion_matrix_un_train0,labeled_recall_train0,labeled_f1_test3,tp_confusion_matrix_un_train3,labeled_brier_train0,labeled_acc_test2,confusion_matrix_lab_test1,tp_confusion_matrix_lab_test2,confusion_matrix_un_train3,labeled_recall_test3,labeled_f1_train3,tn_confusion_matrix_un_train0,fp_confusion_matrix_lab_train2,labeled_acc_train1,fp_confusion_matrix_lab_train3,labeled_f1_test1,labeled_brier_train2,labeled_avg_prec_test4,pu_score_test4,pr_one_unlabeled_train1,assumed_f1beta10_train0,tn_confusion_matrix_lab_test4,fp_confusion_matrix_lab_test1,assumed_f1_train4,labeled_prec_train2,assumed_f1_test1,tn_confusion_matrix_lab_test3,pu_score_train0,tp_confusion_matrix_un_test4,tn_confusion_matrix_lab_train4,confusion_matrix_lab_train4,labeled_recall_test0,labeled_recall_train3,labeled_recall_train1,tp_confusion_matrix_lab_train1,assumed_brier_train0,tp_confusion_matrix_lab_train4,tn_confusion_matrix_un_train1,labeled_acc_train3,labeled_acc_train2,labeled_roc_auc_train2,labeled_roc_auc_test1,tp_confusion_matrix_lab_train3,labeled_prec_train0,tp_confusion_matrix_lab_train2,pr_one_unlabeled_test2,fn_confusion_matrix_un_test4,confusion_matrix_un_train0,assumed_brier_test1,tp_confusion_matrix_lab_test3,confusion_matrix_lab_test4,tp_confusion_matrix_un_train2,tn_confusion_matrix_un_train2,assumed_f1_test0,tp_confusion_matrix_lab_train0,labeled_f1_train1,fp_confusion_matrix_un_train4,labeled_f1_test2,labeled_av

In [28]:
score_labels = set([s[:-1] for s in score_grid.columns])

In [29]:
score_labels

{'assumed_brier_test',
 'assumed_brier_train',
 'assumed_f1_test',
 'assumed_f1_train',
 'assumed_f1beta10_test',
 'assumed_f1beta10_train',
 'confusion_matrix_lab_test',
 'confusion_matrix_lab_train',
 'confusion_matrix_un_test',
 'confusion_matrix_un_train',
 'fn_confusion_matrix_lab_test',
 'fn_confusion_matrix_lab_train',
 'fn_confusion_matrix_un_test',
 'fn_confusion_matrix_un_train',
 'fp_confusion_matrix_lab_test',
 'fp_confusion_matrix_lab_train',
 'fp_confusion_matrix_un_test',
 'fp_confusion_matrix_un_train',
 'labeled_acc_test',
 'labeled_acc_train',
 'labeled_avg_prec_test',
 'labeled_avg_prec_train',
 'labeled_brier_test',
 'labeled_brier_train',
 'labeled_f1_test',
 'labeled_f1_train',
 'labeled_prec_test',
 'labeled_prec_train',
 'labeled_recall_test',
 'labeled_recall_train',
 'labeled_roc_auc_test',
 'labeled_roc_auc_train',
 'pr_one_unlabeled_test',
 'pr_one_unlabeled_train',
 'pu_score_test',
 'pu_score_train',
 'tn_confusion_matrix_lab_test',
 'tn_confusion_matrix_l

In [29]:
for label in score_labels:
    label_score_grid = score_grid[[s for s in score_grid.columns if label == s[:-1]]]
    mean_for_label = label_score_grid.mean(axis=1)
    std_for_label = label_score_grid.std(axis=1)
    score_grid["mean_{}".format(label)] = mean_for_label
    score_grid["std_{}".format(label)] = std_for_label

In [30]:
score_grid.columns

Index(['tp_confusion_matrix_un_train1', 'confusion_matrix_un_train1',
       'fp_confusion_matrix_lab_test0', 'labeled_avg_prec_train3',
       'labeled_avg_prec_test3', 'labeled_recall_train1',
       'confusion_matrix_un_train0', 'confusion_matrix_lab_train0',
       'labeled_recall_train2', 'tn_confusion_matrix_un_train1',
       ...
       'mean_fn_confusion_matrix_un_train', 'std_fn_confusion_matrix_un_train',
       'mean_fp_confusion_matrix_lab_train',
       'std_fp_confusion_matrix_lab_train', 'mean_labeled_avg_prec_test',
       'std_labeled_avg_prec_test', 'mean_labeled_avg_prec_train',
       'std_labeled_avg_prec_train', 'mean_pr_one_unlabeled_train',
       'std_pr_one_unlabeled_train'],
      dtype='object', length=308)

In [31]:
score_grid[[s for s in score_grid.columns if 'mean' in s]]

,mean_confusion_matrix_un_train,mean_tp_confusion_matrix_lab_test,mean_assumed_f1beta10_test,mean_confusion_matrix_lab_train,mean_fp_confusion_matrix_lab_test,mean_tp_confusion_matrix_lab_train,mean_labeled_recall_train,mean_fp_confusion_matrix_un_train,mean_labeled_roc_auc_test,mean_labeled_prec_test,mean_tp_confusion_matrix_un_train,mean_assumed_brier_train,mean_labeled_acc_train,mean_pr_one_unlabeled_test,mean_pu_score_train,mean_fp_confusion_matrix_un_test,mean_assumed_f1beta10_train,mean_assumed_f1_train,mean_tn_confusion_matrix_un_test,mean_labeled_brier_train,mean_tn_confusion_matrix_lab_test,mean_labeled_brier_test,mean_labeled_f1_train,mean_tn_confusion_matrix_un_train,mean_tn_confusion_matrix_lab_train,mean_pu_score_test,mean_fn_confusion_matrix_lab_train,mean_labeled_acc_test,mean_tp_confusion_matrix_un_test,mean_assumed_brier_test,mean_labeled_roc_auc_train,mean_labeled_f1_test,mean_confusion_matrix_lab_test,mean_confusion_matrix_un_test,mean_fn_confusion_matrix_lab_test,mean_labeled_prec_train,mean_assumed_f1_test,mean_labeled_recall_test,mean_fn_confusion_matrix_un_test,mean_fn_confusion_matrix_un_train,mean_fp_confusion_matrix_lab_train,mean_labeled_avg_prec_test,mean_labeled_avg_prec_train,mean_pr_one_unlabeled_train
0,NaN,69.8,0.977495,NaN,2.8,285.6,1.000000,0.0,0.955995,0.961949,285.6,0.004454,1.000000,0.0,1.593836,2.8,1.000000,1.000000,39.6,0.004454,39.6,0.031369,1.000000,169.6,169.6,1.498956,0.0,0.961493,69.8,0.031369,1.000000,0.969571,NaN,NaN,1.6,1.000000,0.969571,0.977660,1.6,0.0,0.0,0.976807,1.000000,0.0
1,NaN,69.6,0.974130,NaN,6.2,279.2,0.977593,23.6,0.914716,0.920952,279.2,0.053017,0.934100,0.0,1.436876,6.2,0.977010,0.949049,36.2,0.053017,36.2,0.059581,0.949049,146.0,146.0,1.429970,6.4,0.929912,69.6,0.059581,0.919228,0.946309,NaN,NaN,1.8,0.922206,0.946309,0.974726,1.8,6.4,23.6,0.955773,0.956928,0.0
2,NaN,69.4,0.971409,NaN,6.0,279.4,0.978297,24.0,0.915708,0.923483,279.4,0.052809,0.933664,0.0,1.435975,6.0,0.977694,0.948740,36.4,0.052809,36.4,0.058745,0.948740,145.6,145.6,1.429882,6.2,0.929942,69.4,0.058745,0.918400,0.946275,NaN,NaN,2.0,0.920944,0.946275,0.971948,2.0,6.2,24.0,0.956519,0.956428,0.0
3,NaN,70.0,0.980302,NaN,2.4,285.6,1.000000,0.0,0.962146,0.967547,285.6,0.004304,1.000000,0.0,1.593836,2.4,1.000000,1.000000,40.0,0.004304,40.0,0.030681,1.000000,169.6,169.6,1.511983,0.0,0.966772,70.0,0.030681,1.000000,0.973785,NaN,NaN,1.4,1.000000,0.973785,0.980438,1.4,0.0,0.0,0.980126,1.000000,0.0
4,NaN,69.6,0.974653,NaN,3.2,285.6,1.000000,4.4,0.949880,0.956635,285.6,0.013440,0.990333,0.0,1.569658,3.2,0.999847,0.992356,39.2,0.013440,39.2,0.033306,0.992356,165.2,165.2,1.486364,0.0,0.956214,69.6,0.033306,0.987027,0.965487,NaN,NaN,1.8,0.984830,0.965487,0.974844,1.8,0.0,4.4,0.973627,0.992415,0.0
5,NaN,69.6,0.974742,NaN,2.8,285.6,1.000000,0.0,0.954551,0.961712,285.6,0.004230,1.000000,0.0,1.593836,2.8,1.000000,1.000000,39.6,0.004230,39.6,0.031730,1.000000,169.6,169.6,1.494246,0.0,0.959723,69.6,0.031730,1.000000,0.968036,NaN,NaN,1.8,1.000000,0.968036,0.974883,1.8,0.0,0.0,0.976170,1.000000,0.0
